In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.preprocessing.text import one_hot

Using TensorFlow backend.


In [ ]:
dfs = []

for df in pd.read_csv('train', chunksize=50000):
    df = pd.concat([df.loc[df['click'] == 0],
                    df.loc[df['click'] == 1].sample(frac=2, replace=True)])
    dfs.append(df.sample(frac=1))
    
pd.concat(np.random.shuffle(dfs)).to_csv('train-sampled.csv', index=False)
del dfs

In [ ]:
d = {'C1':10, 'C14':3000, 'C15':10, 'C16':10, 'C17':500,
     'C18':5, 'C19':70, 'C20':180, 'C21':65}

model = Sequential() 
model.add(Dense(units=2,
                input_dim=sum(d.values())+1,
                activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['sparse_categorical_accuracy'])

for df in pd.read_csv('train-subset.csv', chunksize=50000):
    X = np.hstack([to_categorical(df[col].apply(lambda r: (one_hot(str(r),
                                                                   n_features))
                                               ).tolist(),
                                  n_features)
                   for (col, n_features) in d.items()])

    X = np.hstack([X, df['banner_pos'].values.reshape(-1, 1)])
    
    model.fit(x=X,
              y=df['click'].values,
              epochs=5,
              validation_split=0.2)